In [68]:
from dataset import Dataset

#### DEFINE PARAMETERS

In [70]:
TICKERS = ['BTCUSDT', 'ETHUSDT', 'LTCUSDT',
           'BNBUSDT', 'ADAUSDT', 'XRPUSDT', 'DOTUSDT']
DAYS = 30
TIMEFRAME = '5m'
SPLIT = 0.8

#### Download Dataset

In [27]:
dataset = Dataset().get_datas(days=DAYS, tickers=TICKERS, ts=TIMEFRAME)

#### Check Dataset

In [45]:
data = dataset.copy()
data['dayOfYear'] = data.index.dayofyear
data.groupby('symbol')\
    .apply(lambda x: x.tail(2))

open        high         low       close  \
symbol  time                                                                  
ADAUSDT 2022-04-19 07:20:00      0.9290      0.9300      0.9280      0.9290   
        2022-04-19 07:25:00      0.9280      0.9290      0.9270      0.9290   
BNBUSDT 2022-04-19 07:20:00    420.9000    421.0000    418.2000    418.9000   
        2022-04-19 07:25:00    418.9000    419.1000    418.4000    419.1000   
BTCUSDT 2022-04-19 07:20:00  40666.8500  40666.8600  40571.0000  40630.0100   
        2022-04-19 07:25:00  40630.0100  40630.0100  40598.1000  40629.7000   
DOTUSDT 2022-04-19 07:20:00     18.1400     18.1500     18.0800     18.1000   
        2022-04-19 07:25:00     18.0900     18.1200     18.0800     18.1200   
ETHUSDT 2022-04-19 07:20:00   3041.6300   3041.7600   3030.3400   3033.9700   
        2022-04-19 07:25:00   3033.9700   3035.2300   3032.7000   3034.0800   
LTCUSDT 2022-04-19 07:20:00    111.0000    111.1000    110.6000    110.7000   
        2022-04-19 07:25:00    110.7000    110.8000    110.6000    110.7000   
XRPUSDT 2022-04-19 07:20:00      0.7666      0.7666      0.7635      0.7636   
        2022-04-19 07:25:00      0.7635      0.7649      0.7635      0.7647   

                                   volume   symbol  dayOfYear  
symbol  time                                                   
ADAUSDT 2022-04-19 07:20:00  6.586054e+05  ADAUSDT        109  
        2022-04-19 07:25:00  1.209391e+05  ADAUSDT        109  
BNBUSDT 2022-04-19 07:20:00  5.333802e+03  BNBUSDT        109  
        2022-04-19 07:25:00  1.262911e+03  BNBUSDT        109  
BTCUSDT 2022-04-19 07:20:00  2.095408e+02  BTCUSDT        109  
        2022-04-19 07:25:00  8.743117e+01  BTCUSDT        109  
DOTUSDT 2022-04-19 07:20:00  1.084790e+04  DOTUSDT        109  
        2022-04-19 07:25:00  3.130990e+03  DOTUSDT        109  
ETHUSDT 2022-04-19 07:20:00  2.500852e+03  ETHUSDT        109  
        2022-04-19 07:25:00  6.907925e+02  ETHUSDT        109  
LTCUSDT 2022-04-19 07:20:00  3.028465e+03  LTCUSDT        109  
        2022-04-19 07:25:00  2.670190e+02  LTCUSDT        109  
XRPUSDT 2022-04-19 07:20:00  1.724950e+06  XRPUSDT        109  
        2022-04-19 07:25:00  4.664260e+05  XRPUSDT        109

#### Processing Dataset

In [72]:
df = data.groupby('symbol')\
    .apply(lambda x: x)\
    .assign(target=lambda x: x['close'].shift(-1))\
    .groupby('dayOfYear')\
    .apply(lambda x: x)\
    .dropna()
print(df.head(len(TICKERS)))

                           open       high         low       close  \
time                                                                 
2022-03-20 00:05:00      0.9020      0.908      0.9010      0.9060   
2022-03-20 00:05:00    399.9000    400.100    399.3000    400.0000   
2022-03-20 00:05:00  42201.1300  42246.280  42172.1500  42227.7100   
2022-03-20 00:05:00     19.4000     19.440     19.3600     19.4300   
2022-03-20 00:05:00   2950.8500   2959.290   2949.1000   2957.3800   
2022-03-20 00:05:00    116.8000    117.500    116.7000    117.3000   
2022-03-20 00:05:00      0.8195      0.822      0.8186      0.8203   

                           volume   symbol  dayOfYear      target  
time                                                               
2022-03-20 00:05:00  783664.00000  ADAUSDT         79      0.9060  
2022-03-20 00:05:00    2993.00400  BNBUSDT         79    400.1000  
2022-03-20 00:05:00     103.34117  BTCUSDT         79  42214.7100  
2022-03-20 00:05:00   13952.6

### Split Dataset on train and test data

In [76]:

unique_days = df.dayOfYear.unique()
train_days = unique_days[:int(len(unique_days) * SPLIT)]
test_days = unique_days[int(len(unique_days) * SPLIT):]
train = df[df.dayOfYear.isin(train_days)]
test = df[df.dayOfYear.isin(test_days)]

In [77]:
X_train = train.drop(['target', 'dayOfYear', 'symbol'], axis=1)
y_train = train['target']
X_test = test.drop(['target', 'dayOfYear', 'symbol'], axis=1)
y_test = test['target']
print('train days:', X_train.values.shape)
print('test days:', X_test.values.shape)

train days: (48377, 5)
test days: (12725, 5)
